In [1]:
import os
import os.path
import sys
d = os.path.join(os.getcwd(), '..')
print(d)
sys.path.append(d)

C:\Users\Zachary\Documents\GitHub\COMP 3201 - TSP Evolutionary Algorithm\src\..


In [2]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure as Figure
import seaborn as sns
import numpy as np
import pandas as pd

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

# https://pandas.pydata.org/pandas-docs/stable/visualization.html
# http://pandas.pydata.org/pandas-docs/version/0.13/visualization.html
# https://matplotlib.org/users/pyplot_tutorial.html

# Set up the style of the graphs, and display a sample graph
graph_style = 5
plt.style.use(plt.style.available[graph_style])  # 5, 14, 22
sns.set_context("paper")

In [3]:
# Standard Imports

# These should not be changed, as they are used to set the 'constant'
# global variables for each module.
from EA_Methods.Pandas_Rep import ParentSelectionMethods as PSM
from EA_Methods.Pandas_Rep import RecombinationMethods as RM
from EA_Methods.Pandas_Rep import MutationMethods as MM
from EA_Methods.Pandas_Rep import SurvivorSelectionMethods as SSM


# Modular Imports

# Modular Problem Definition
# By changing these imports, you can easily redifine the problem you are attempting
# to solve, without having to re-write much of the code.
from Setups.TSP import TSP_PDS as DEF
from Setups.TSP.TSP_PDS import read_tsp_file as parse_file


# Modular Function Definitions
# By swapping out which functions are imported, but keeping the aliases,
# the entire structure of the EA algorithm can be changed without rewriting code
# in the main method.
from Setups.TSP.TSP_PDS import random_initialization as initialize #= DEF.heurisitic_initialization
from Setups.TSP.TSP_PDS import euclidean_distance as eval_fitness #= DEF.euclidean_distance
from EA_Methods.Pandas_Rep.ParentSelectionMethods import roulette as parent_selection #= PSM.
from EA_Methods.Pandas_Rep.RecombinationMethods import recombination_cut_crossover as generate_offspring #= RM.
from EA_Methods.Pandas_Rep.MutationMethods import permutation_swap as apply_mutation #= MM.
from EA_Methods.Pandas_Rep.SurvivorSelectionMethods import replacement as select_survivors # = SSM.


# Global Variable Initialization
FILENUM             =      1  # Changing this variable changes which file to parse.
genome_length       =      parse_file(FILENUM)
generation_limit    =      10000
population_size     =      60
mating_pool_size    =      population_size//2 if (population_size//2) % 2 == 0 else (population_size//2)+1  # has to be even

tournament_size     =      population_size//10
crossover_rate      =      0.9
crossover_point     =      genome_length//3
mutation_rate       =      0.2


# Key Variable Setters
# These functions typically do not need to be changed, and simply
# allow easy changing of all rates and variables from this cell.
PSM.set_tournament_size(tournament_size)
RM.set_genome_length(genome_length)
RM.set_crossover_point(crossover_point)
RM.set_crossover_rate(crossover_rate)
MM.set_genome_length(genome_length)
MM.set_mutation_rate(mutation_rate)
MM.set_fitness_function(eval_fitness)
SSM.set_population_size(population_size)
SSM.set_mating_pool_size(mating_pool_size)
DEF.set_fitness_function(eval_fitness)

DEF.CITIES.head()

,Longitude (Range shifted),Latitude (Range shifted)
City,,
0,-3314.58335,-3633.33335
1,-3247.91665,-3600.00005
2,-2847.91665,449.99995
3,-2547.91665,-683.33335
4,-2547.91665,-1500.00005


In [4]:
PSM.set_op(min)
SSM.set_op(min)

# Initialize Population
population = initialize(population_size, genome_length)
population

,individuals,fitnesses
indexes,,
0,"[5, 26, 6, 9, 25, 13, 19, 17, 15, 22, 0, 7, 2,...",104690.018553
1,"[17, 23, 4, 20, 7, 14, 10, 21, 0, 6, 24, 9, 15...",127914.689391
2,"[24, 3, 23, 28, 19, 27, 21, 22, 8, 0, 10, 11, ...",93684.218979
3,"[5, 18, 25, 6, 27, 3, 2, 28, 24, 21, 0, 23, 16...",120982.941884
4,"[23, 27, 20, 13, 7, 18, 4, 17, 12, 9, 26, 10, ...",113246.243765
5,"[26, 7, 14, 16, 9, 22, 1, 3, 20, 0, 11, 28, 17...",112263.666595
6,"[18, 25, 4, 26, 11, 24, 16, 22, 1, 8, 12, 14, ...",109816.661648
7,"[2, 3, 8, 9, 20, 1, 28, 19, 7, 13, 17, 22, 11,...",118988.602815
8,"[22, 9, 14, 19, 15, 20, 26, 17, 7, 24, 13, 21,...",100018.293280


In [13]:
parents = parent_selection(population, mating_pool_size)
parents

,individuals,fitnesses
indexes,,
28,"[18, 8, 0, 26, 3, 23, 22, 12, 11, 6, 16, 25, 2...",105627.379403
49,"[9, 0, 8, 5, 25, 14, 2, 6, 15, 21, 11, 19, 26,...",106841.405238
44,"[11, 19, 1, 26, 15, 0, 8, 3, 4, 21, 5, 6, 27, ...",117171.739794
54,"[28, 1, 17, 10, 18, 13, 22, 12, 9, 7, 0, 2, 3,...",106231.451158
21,"[27, 25, 0, 22, 21, 3, 1, 7, 23, 11, 20, 12, 1...",125257.810270
55,"[11, 1, 7, 10, 2, 25, 28, 4, 17, 24, 14, 16, 2...",112685.342180
48,"[23, 26, 28, 22, 24, 20, 15, 10, 6, 18, 17, 3,...",90987.249504
45,"[24, 0, 7, 26, 17, 25, 6, 15, 20, 13, 1, 5, 14...",115953.783614
37,"[20, 23, 1, 11, 3, 18, 10, 4, 13, 8, 17, 19, 1...",114082.880597


In [67]:
offspring = generate_offspring(parents)
offspring.head()

,individuals,fitnesses
indexes,,
0,"[3, 10, 16, 11, 19, 7, 5, 8, 6, 21, 23, 17, 12...",103281.723296
1,"[11, 2, 5, 25, 21, 14, 23, 28, 20, 10, 18, 22,...",109016.441832
2,"[10, 19, 22, 4, 11, 15, 24, 8, 21, 27, 7, 1, 6...",103005.152389
3,"[9, 0, 4, 13, 5, 3, 11, 17, 7, 2, 15, 16, 19, ...",95141.853474
4,"[9, 12, 23, 3, 21, 1, 4, 17, 22, 14, 15, 10, 6...",103617.514271


In [69]:
test = np.array([x for x in range(10)])
test[2], test[5] = test[5], test[2]
test

array([0, 1, 5, 3, 4, 2, 6, 7, 8, 9])

In [41]:
def evolution_algorithm(maximize, print_gens=True, display_freq=None):
    if display_freq is None: display_freq = generation_limit
    
    # Setting dynamic standard for 'best'
    op = max if maximize else min
    PSM.set_op(op)
    SSM.set_op(op)

    # Initialize Population
    population = initialize(population_size, genome_length)
    
    if print_gens:
        # TODO - Better handle demonstration output.
        DEF.start_up_display()

    # Evolution starts
    for generation in range(generation_limit):

        # Generation Info
        if print_gens and (generation % display_freq == display_freq - 1):
            # TODO - Better handle demonstration output.
            op_fit = op(fitness)
            optimal_solutions = [i + 1 for i in range(population_size) if fitness[i] == op_fit]
            #TSP.generation_display(optimal_solutions[0])
            print("Generation: {}\n - Best fitness: {}\n - Avg. fitness: {}\n - Number of optimal solutions: {}/{}\n".format(
                generation+1, op(fitness), sum(fitness)/len(fitness), len(optimal_solutions), population_size)
            )

        # Parent Selection
        parents = parent_selection(fitness, mating_pool_size)

        # Generate Offspring (Recombination)
        offspring = generate_offspring(parents)

        # Mutation Application
        offspring = apply_mutation(offspring)

        # Survivor Selection
        population = select_survivors(population, offspring)
        
        # df.sample(frac=1).reset_index(drop=True)
    # Evolution ends
        
    # Final Fitness Info
    op_fit = op(fitness)
    optimal_solutions = [i + 1 for i in range(population_size) if fitness[i] == op_fit]
    print("\nBest solution fitness:", op_fit, "\nNumber of optimal solutions: ", len(optimal_solutions), '/', population_size)
    print("Best solution indexes:\n", optimal_solutions)
    print('\n\n\n\n\n\n')

In [ ]:
freq = min(int(generation_limit * 0.1), 500)
print('\n\nDisplaying information every {} generations.\n\n'.format(freq))
evolution_algorithm(False, True, freq)

## Helper Functions

In [ ]:
# Sahara
parse_file(1)
DEF.start_up_display()
#TSP.brute_force_solver()

In [ ]:
# Uraguay
parse_file(2)
DEF.start_up_display()
#TSP.brute_force_solver()

In [ ]:
# Canada
parse_file(3)
DEF.start_up_display()
#TSP.brute_force_solver()